## **Define ASR Model**

**TODO**

- Change input to take HF instead of wav files
- Save finalized (predicted) transcriptions to a list and the ground truths to a list
  - Clear ```current_transcription``` after the file is done
  - Add its contents to ```predicted_transcriptions```
- Go to next file once the live transcription is done (Break out of while loop)
- Calculate WER

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!pip uninstall -y torch torchvision torchaudio triton nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-nccl-cu12

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
Found existing installation: nvidia-cublas-cu12 12.5.3.2
Uninstalling nvidia-cublas-cu12-12.5.3.2:
  Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
  Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
Found existing installation: nvidia-cudnn-cu12 9.3.0.75
Uninstalling nvidia-cudnn-cu12-9.3.0.75:
  Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
Found exi

In [3]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu125

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu125
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
import triton
import torch.backends.cudnn as cudnn

print("Torch Version:", torch.__version__)
print("Torch CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())
print("Triton Version:", triton.__version__)
print("cuDNN Version:", cudnn.version())

Torch Version: 2.6.0+cu124
Torch CUDA Version: 12.4
CUDA Available: True
Triton Version: 3.2.0
cuDNN Version: 90100


<br/>
<br/>
<br/>

In [5]:
!pip install -q datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00


**User Action Required**
- Define model to use

In [6]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from google.colab import drive
import time

# Google Drive Env Setup
whisper_ver = 'whisper-base'
checkpoint_num = '5400'
drive.mount('/content/drive')
checkpoint_path = f'/content/drive/My Drive/{whisper_ver}-noiseaugmented-continued-checkpoints/checkpoint-{checkpoint_num}'

# Model setup code for fine-tuned whisper
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = WhisperForConditionalGeneration.from_pretrained(checkpoint_path).to(device)
model.config.use_cache = True
processor = WhisperProcessor.from_pretrained(f"openai/{whisper_ver}", language="en", task="transcribe")

Mounted at /content/drive


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

## **Define VAD model-related dependencies**

In [7]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00


In [8]:
import torch
import warnings
import numpy as np
import onnxruntime


class OnnxWrapper():
    '''
    Code taken from: https://github.com/snakers4/silero-vad/blob/master/src/silero_vad/utils_vad.py

    See https://github.com/collabora/WhisperLive/blob/main/whisper_live/vad.py
    '''
    def __init__(self, path, force_onnx_cpu=False):
        opts = onnxruntime.SessionOptions()
        opts.inter_op_num_threads = 1
        opts.intra_op_num_threads = 1

        if force_onnx_cpu and 'CPUExecutionProvider' in onnxruntime.get_available_providers():
            self.session = onnxruntime.InferenceSession(path, providers=['CPUExecutionProvider'], sess_options=opts)
        else:
            self.session = onnxruntime.InferenceSession(path, sess_options=opts)

        self.reset_states()
        if '16k' in path:
            warnings.warn('This model support only 16000 sampling rate!')
            self.sample_rates = [16000]
        else:
            self.sample_rates = [8000, 16000]

    def _validate_input(self, x, sr: int):
        if x.dim() == 1:
            x = x.unsqueeze(0)
        if x.dim() > 2:
            raise ValueError(f"Too many dimensions for input audio chunk {x.dim()}")

        if sr != 16000 and (sr % 16000 == 0):
            step = sr // 16000
            x = x[:,::step]
            sr = 16000

        if sr not in self.sample_rates:
            raise ValueError(f"Supported sampling rates: {self.sample_rates} (or multiply of 16000)")
        if sr / x.shape[1] > 31.25:
            raise ValueError("Input audio chunk is too short")

        return x, sr

    def reset_states(self, batch_size=1):
        self._state = torch.zeros((2, batch_size, 128)).float()
        self._context = torch.zeros(0)
        self._last_sr = 0
        self._last_batch_size = 0

    def __call__(self, x, sr: int):

        x, sr = self._validate_input(x, sr)
        num_samples = 512 if sr == 16000 else 256

        if x.shape[-1] != num_samples:
            raise ValueError(f"Provided number of samples is {x.shape[-1]} (Supported values: 256 for 8000 sample rate, 512 for 16000)")

        batch_size = x.shape[0]
        context_size = 64 if sr == 16000 else 32

        if not self._last_batch_size:
            self.reset_states(batch_size)
        if (self._last_sr) and (self._last_sr != sr):
            self.reset_states(batch_size)
        if (self._last_batch_size) and (self._last_batch_size != batch_size):
            self.reset_states(batch_size)

        if not len(self._context):
            self._context = torch.zeros(batch_size, context_size)

        x = torch.cat([self._context, x], dim=1)
        if sr in [8000, 16000]:
            ort_inputs = {'input': x.numpy(), 'state': self._state.numpy(), 'sr': np.array(sr, dtype='int64')}
            ort_outs = self.session.run(None, ort_inputs)
            out, state = ort_outs
            self._state = torch.from_numpy(state)
        else:
            raise ValueError()

        self._context = x[..., -context_size:]
        self._last_sr = sr
        self._last_batch_size = batch_size

        out = torch.from_numpy(out)
        return out

    def audio_forward(self, x, sr: int):
        outs = []
        x, sr = self._validate_input(x, sr)
        self.reset_states()
        num_samples = 512 if sr == 16000 else 256

        if x.shape[1] % num_samples:
            pad_num = num_samples - (x.shape[1] % num_samples)
            x = torch.nn.functional.pad(x, (0, pad_num), 'constant', value=0.0)

        for i in range(0, x.shape[1], num_samples):
            wavs_batch = x[:, i:i+num_samples]
            out_chunk = self.__call__(wavs_batch, sr)
            outs.append(out_chunk)

        stacked = torch.cat(outs, dim=1)
        return stacked.cpu()




class VoiceActivityDetector:
    '''
    See https://github.com/collabora/WhisperLive/blob/main/whisper_live/vad.py

    Onxx model:
    '''
    def __init__(self, path, threshold=0.5, frame_rate=16000):
        """
        Initializes the VoiceActivityDetector with a voice activity detection model and a threshold.

        Args:
            threshold (float, optional): The probability threshold for detecting voice activity. Defaults to 0.5.
        """
        self.model = OnnxWrapper(path=path,force_onnx_cpu=True)
        self.threshold = threshold
        self.frame_rate = frame_rate

    def __call__(self, audio_frame):
        """
        Determines if the given audio frame contains speech by comparing the detected speech probability against
        the threshold.

        Args:
            audio_frame (np.ndarray): The audio frame to be analyzed for voice activity. It is expected to be a
                                      NumPy array of audio samples.

        Returns:
            bool: True if the speech probability exceeds the threshold, indicating the presence of voice activity;
                  False otherwise.
        """
        speech_probs = self.model.audio_forward(torch.from_numpy(audio_frame.copy()), self.frame_rate)[0]
        return torch.any(speech_probs > self.threshold).item()

## **Load HF Datasets**

**User Action Required**

- Define the number of test samples

In [9]:
from datasets import load_dataset
dataset_repo_test = "johnlohjy/imda_nsc_p3_test_noiseaugmented"
dataset_test = load_dataset(dataset_repo_test, split='test', streaming=True, trust_remote_code=True)

imda_nsc_p3_test_noiseaugmented.py:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

In [10]:
num_test_examples = 5 #1000

In [11]:
from itertools import islice
from torch.utils.data import IterableDataset

class SlicedDataset(IterableDataset):
    def __init__(self, dataset, num_examples):
        self.dataset = dataset
        self.num_examples = num_examples

    def __iter__(self):
        return islice(iter(self.dataset), self.num_examples)

    def __len__(self):
        return self.num_examples

dataset_test_reduced = SlicedDataset(dataset_test, num_examples=num_test_examples)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Loop to simulate live transcription (from a HuggingFace Datasets)**

## **Define Server-Related Classes for ```HF Dataset```**

In [12]:
import json
import numpy as np
import os
import wave
import threading
import textwrap
from IPython.display import clear_output

# Helper function Provided by Alexander Veysov
# See
# - https://github.com/snakers4/silero-vad/blob/master/examples/pyaudio-streaming/pyaudio-streaming-examples.ipynb
# Or (different implementations)
# - https://github.com/snakers4/silero-vad/blob/master/examples/microphone_and_webRTC_integration/microphone_and_webRTC_integration.py
def int2float(sound):
    abs_max = np.abs(sound).max()
    sound = sound.astype('float32')
    if abs_max > 0:
        sound *= 1/32768
    sound = sound.squeeze()  # depends on the use case
    return sound


class Client:
    def __init__(self, transcriber, transcriber_processor, RATE=16000, buffer_limit=45, finalized_seg_len=15, lagging_cap=25):
        self.frames_np = None # Store frames buffer as a numpy array
        self.frames_offset = 0.0 # Track frames offset from the very start (global timing). Tracks the duration of audio discarded
        self.timestamp_offset = 0.0 # Track transcription offset from the very start (global timing). Tracks the start of the transcription
        self.transcriber = transcriber # Initialize Whisper ASR model
        self.transcriber_processor = transcriber_processor # Initialize Whisper ASR processor
        self.transcript = [] # Store fully transcribed segments
        self.lock = threading.Lock() # for shared resources: frames_np. https://realpython.com/python-thread-lock/#threadinglock-for-primitive-locking
        self.RATE = RATE

        # Thread to run speech-to-text function
        '''
        Main "Entry point" for client
        '''
        self.running = True
        self.trans_thread = threading.Thread(target=self.speech_to_text)
        self.trans_thread.start()

        self.buffer_limit = buffer_limit # Max length of audio we will store in the RAM
        self.finalized_seg_len = finalized_seg_len # Length of each 'finalized' segment in seconds
        self.lagging_cap = lagging_cap # Max allowabe seconds of audio to transcribe before updating timestamp_offset -> Prevent lag because Whisper may take too long on longer audio
        self.eos = False # End-Of-Speech Flag for super long silence

    def add_frames(self, frame_np):
        self.lock.acquire()
        # If the buffer is more than buffer_limit (seconds)
        if self.frames_np is not None and self.frames_np.shape[0] > self.buffer_limit*self.RATE:
            # Save RAM:
            # Increase frames_offset by 30s
            # Discard oldest 30s of audio from buffer
            self.frames_offset += 30.0
            self.frames_np = self.frames_np[int(30*self.RATE):]
            # Update timestamp_offset if its less than the frames_offset
            if self.timestamp_offset < self.frames_offset:
                self.timestamp_offset = self.frames_offset

        # If the frame buffer is empty, initialise it with the new audio frames
        if self.frames_np is None:
            self.frames_np = frame_np.copy()
        # Else, append the new audio chunk to the existing buffer
        else:
            self.frames_np = np.concatenate((self.frames_np, frame_np), axis=0)
        # unlock the critical section of code above
        self.lock.release()


    def speech_to_text(self):
        while self.running:
            # Wait for some chunks to arrive
            if self.frames_np is None:
                time.sleep(0.02)
                continue

            # Adjust the timestamp_offset/transcription tracker too much audio for Whisper to transcribe (long processing time),
            # resulting in laggy transcriptions
            self.clip_audio_if_no_valid_segment()

            # Get the audio to be transcribed using the timestamp_offset/transcription tracker
            input_bytes, duration = self.get_audio_chunk_for_processing()

            # Prevent hallucination on too short audio
            if duration < 0.4:
                continue

            # Transcribe the audio
            input_sample = input_bytes.copy()
            self.transcribe_audio(input_sample, duration)


    def clip_audio_if_no_valid_segment(self):
        '''
        If there is > {lagging_cap} seconds of audio to transcribe in the buffer,
        adjust timestamp_offset such that its only 5s behind the total audio added so far

        If there is <={lagging_cap} seconds of audio to transcribe, its okay

        self.timestamp_offset - self.frames_offset = Audio in the buffer that has already been transcribed
        '''
        with self.lock:
            # If there is more than {lagging_cap} seconds of audio to transcribe
            if self.frames_np[int((self.timestamp_offset - self.frames_offset)*self.RATE):].shape[0] > self.lagging_cap * self.RATE:
                # Adjust timestamp_offset s.t it is only 5s behind the total audio added so far
                duration = self.frames_np.shape[0] / self.RATE
                self.timestamp_offset = self.frames_offset + duration - 5


    def get_audio_chunk_for_processing(self):
        '''
        Get the audio to be transcribed from the buffer

        self.timestamp_offset - self.frames_offset = Audio in the buffer that has already been transcribed
        '''
        with self.lock:
            samples_take = max(0, (self.timestamp_offset - self.frames_offset) * self.RATE)
            input_bytes = self.frames_np[int(samples_take):].copy()
        duration = input_bytes.shape[0] / self.RATE
        return input_bytes, duration


    def transcribe_audio(self, input_bytes, duration):
        input_features = self.transcriber_processor.feature_extractor(input_bytes, sampling_rate=16000).input_features[0]
        input_features = torch.tensor(input_features).unsqueeze(0).to(device)
        generated_ids = self.transcriber.generate(input_features)
        last_segment = self.transcriber_processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Print the transcriptions to be printed and handle update of timestamp_offset as required
        self.handle_transcription_output(last_segment, duration)


    def handle_transcription_output(self, last_segment, duration):
        self.send_transcription_to_client({"text": last_segment})

        if self.is_segment_complete(duration):
            self.update_timestamp_offset(last_segment, duration)
            # Add the finalized segment (finalized with full context) to the global current predictions variable
            current_prediction.append(last_segment)


    def is_segment_complete(self, duration):

        # FOR TESTING PURPOSES maybe need an or condition because what if the remaining audio we are transcribing from the file does not hit the
        # specified length of each 'finalized' segment
        # Or maybe its needed in the case where there is really super long silence


        # If the duration of audio we are transcribing is more than the specified length of each 'finalized' segment, return true i.e. complete
        # or there is super long silence
        if duration >= self.finalized_seg_len or self.get_eos():
            return True
        return False


    def send_transcription_to_client(self, segments):
        '''
        Print the transcription
        '''
        text = [segments["text"]]

        '''
        For cmd line like outputs
        os.system("cls" if os.name == "nt" else "clear")
        '''
        clear_output(wait=True) # Delays clearing the output until new output is available -> reduce flickering

        """Prints formatted transcript text."""
        wrapper = textwrap.TextWrapper(width=60)
        for line in wrapper.wrap(text="".join(text)):
            print(line)


    def update_timestamp_offset(self, last_segment, duration):
        # Update the timestamp_offset as this portion of the buffer has been finalized
        # (doesn't need to be transcribed anymore)
        with self.lock:
            self.timestamp_offset += duration

    def get_eos(self):
        self.lock.acquire()
        eos_val = self.eos
        self.lock.release()
        return eos_val

    def set_eos(self, eos):
        self.lock.acquire()
        self.eos = eos
        self.lock.release()

    def stop(self):
        '''
        Signals the thread to stop gracefully.
        '''
        print("Stopping transcription thread...")
        self.running = False
        if self.trans_thread.is_alive():
            self.trans_thread.join()  # Wait for the thread to finish




class ClientManager:
    '''
    Custom client manager class to handle clients
    "connected over the WebSocket server" (not necessarily)
    '''
    def __init__(self):
        self.clients = {}

    def add_client(self, websocket, client):
        '''
        Add a WebSocket server connection info (or just a key) and its associated client
        '''
        self.clients[websocket] = client

    def get_client(self, websocket):
        '''
        Retrieve a client associated with the WebSocket server connection info (or just a key) provided
        '''
        if websocket in self.clients:
            return self.clients[websocket]
        return False




class Server:
    def __init__(self, vad_detector, no_voice_activity_threshold=20):
        '''
        In charge of receiving, checking VA and passing audio to client
        '''
        self.client_manager = None
        self.vad_detector = vad_detector # VAD detector

        self.no_voice_activity_chunks = 0
        self.no_voice_activity_threshold = no_voice_activity_threshold


    def recv_audio(self, websocket, audio_array):
        """
        DEPRECATED entry point for transcription
        - kept in case of conversion to websockets
        Handle the new connection
        Continously process audio frames
        """
        # Handle the new connection
        if not self.handle_new_connection(websocket):
            return

        # Continously process audio frames
        while True:
            if not self.process_audio_frames(websocket, audio_array):
                break


    def handle_new_connection(self, websocket, transcriber, transcriber_processor, RATE=16000, buffer_limit=45, finalized_seg_len=15, lagging_cap=25):
        '''
        Initialise the client manager
        Initialise the new client and add it to the client manager
        '''

        # Initialise the client manager if not done
        if self.client_manager is None:
            self.client_manager = ClientManager()

        # Initialise the new client and add it to the client manager.
        # The websocket acts as the key, or just use any other key
        self.initialize_client(websocket, transcriber, transcriber_processor, RATE, buffer_limit, finalized_seg_len, lagging_cap)

        return True


    def initialize_client(self, websocket, transcriber, transcriber_processor, RATE=16000, buffer_limit=45, finalized_seg_len=15, lagging_cap=25):
        '''
        Initialize the new client and add it to the client manager
        '''
        # Initialize the new client
        client = Client(transcriber, transcriber_processor, RATE, buffer_limit, finalized_seg_len, lagging_cap)

        # Add the client to the client manager
        self.client_manager.add_client(websocket, client)


    def process_audio_frames(self, websocket, audio_array):
        # Get the audio chunk as a numpy array
        frame_np, audio_float32 = self.get_audio_from_websocket(websocket, audio_array)

        # Get the client using its associated key
        client = self.client_manager.get_client(websocket)

        # Check for voice activity in the audio chunk
        voice_active = self.voice_activity(websocket, audio_float32)

        # If there is voice activity, add audio chunk to the client's buffer
        if voice_active:
            client.add_frames(frame_np)

            self.no_voice_activity_chunks = 0
            client.set_eos(False)

        return True


    def get_audio_from_websocket(self, websocket, frame_data):
        '''
        Receive audio chunks and create a numpy array out of it
        '''
        # Creates numpy array without copying it (more efficient)
        # MAKE SURE DATA FORMATS ARE CORRECT
        # VAD:
        # - https://github.com/snakers4/silero-vad/blob/master/examples/pyaudio-streaming/pyaudio-streaming-examples.ipynb
        # - https://github.com/snakers4/silero-vad/blob/master/examples/microphone_and_webRTC_integration/microphone_and_webRTC_integration.py

        return ( frame_data, frame_data )


    def voice_activity(self, websocket, frame_np, no_voice_activity_threshold=20):
        # If there is no voice activity in the audio chunk return False, else return True
        if not self.vad_detector(frame_np):

            self.no_voice_activity_chunks += 1
            if self.no_voice_activity_chunks > self.no_voice_activity_threshold:
                client = self.client_manager.get_client(websocket)
                client.set_eos(True)
                time.sleep(0.1) # Sleep 100m; wait some voice activity

            return False
        return True

    def shutdown(self):
        '''
        Stops all client threads before shutting down the server.
        '''
        print("Stopping all clients")
        for websocket, client in self.client_manager.clients.items():
            client.stop()
        print("Server shutdown complete.")

In [13]:
### SETTINGS ###
chunk = 4096 # number of samples to take per read
RATE = 16000 # number of samples taken per second
'''
chunk duration = chunk_size/sampling_rate
number of chunks in a single second of data = sampling_rate/chunk_size
4096/16000 = 0.256s
'''

buffer_limit=45 # Max length of audio we will store in the RAM
finalized_seg_len=15 # Length of each 'finalized' segment in seconds
lagging_cap=25 # Max allowabe seconds of audio to transcribe before updating timestamp_offset -> Prevent lag because Whisper may take too long on longer audio

vad_model_path = './silero_vad.onnx'
vad_threshold = 0.5
vad_detector = VoiceActivityDetector(vad_model_path,vad_threshold,RATE) # Load the Silero VAD model
no_voice_activity_threshold=20
### SETTINGS ###



# Initialise Server
server = Server(vad_detector,no_voice_activity_threshold)
server.handle_new_connection("websocket",model,processor,RATE, buffer_limit, finalized_seg_len, lagging_cap) # manual call. Just use "websocket" as a key.

# Simulation delay
simulation_delay = chunk/RATE

# Initialize list to store all predicted transcriptions. Global variable
predictions = []
# Initialize list to store current transcriptions. Global variable
current_prediction = []
# Initialize list to store ground truths. Global variable
references = []
# Initialize total audio length
total_audio_len = 0
# Initialize list to store wav fnames to extract SNR values later
wav_fnames = []

# count_example = 0

# FOR TESTNG PURPOSES consider concatenating X number of samples of audio together(?)

for sample in dataset_test_reduced:
  # count_example+=1
  # Get audio samples data
  # Convert from float 64 to float 32
  audio_samples = sample["audio"]["array"].astype(np.float32)

  # Add no_voice_activity_threshold chunks of silence so lastest segment of audio will be saved
  silence = np.zeros(chunk*(no_voice_activity_threshold+5), dtype=np.float32)
  audio_samples = np.concatenate((audio_samples, silence))

  # Get number of samples
  num_audio_samples = len(audio_samples)

  # Read {chunk} samples at a time
  samples_read = 0

  while samples_read < num_audio_samples:

    if samples_read + chunk > num_audio_samples:
      data = audio_samples[samples_read:]
      # if the remaining audio array is < 512 samples (required by SileroVAD)
      if len(data) < 512:
        data = np.concatenate((data, np.zeros(512-len(data), dtype=np.float32)))
    else:
      data = audio_samples[samples_read:samples_read+chunk]

    # Update the number of samples read
    samples_read += len(data)
    # Simulate the time the chunk takes to be 'spoken'
    time.sleep(simulation_delay)
    # Directly call process_audio_frames with the new audio chunk in bytes
    server.process_audio_frames("websocket", data)

  # Wait for processing code to fully finish running
  time.sleep(5)

  predictions.append(' '.join(current_prediction).strip())
  references.append(sample["sentence"])
  print('')
  print('Prediction: ' + ' '.join(current_prediction).strip())
  print('Reference : ' + sample['sentence'])
  print('')
  current_prediction = []
  total_audio_len += num_audio_samples/16000
  wav_fnames.append(os.path.splitext(sample["path"].split('/')[-1])[0])

# Shutdown the server
server.shutdown()

# Calculate total length of audio as well

weird event okay

Prediction: weird event okay
Reference : weird event okay

Stopping all clients
Stopping transcription thread...
Server shutdown complete.


In [15]:
for i in range(len(predictions)):
  print(predictions[i])
  print(references[i])
  print('')

you know ya she was from france she had shared to me about her culture so uh she would she want to learn and she would when i would visit lah when i travel the house looking forward to that it's only not interesting generation gap what's the biggest internalality or difference between your generation and your parent's generation okay
you know yeah she was from france so that's she shared with me about her culture lah so you know err she would show me around when she would when i would visit lah when i travel to france looking forward to that it's really not interesting generation gap what's the biggest similarity or difference between your generation and your parent's generation okay

would you say it's more positive or negative neutral lah good for the country in uh in having as in globalization is good for the country okay okay never ya okay next tekam tekam how do you choose four d or total numbers mm okay
would you say it's more positive or negative neutral lah good for the country

**Calculate WER**

In [16]:
!pip install evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 94.6 MB/s eta 0:00:00


In [23]:
import evaluate
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
metric = evaluate.load("wer")
normalizer = BasicTextNormalizer()

In [24]:
predictions = [normalizer(pred) for pred in predictions]
references = [normalizer(ref) for ref in references]

In [25]:
wer = 100 * metric.compute(predictions=predictions, references=references)

In [26]:
wer

24.09090909090909

In [27]:
total_audio_len

129.477125

In [29]:
total_audio_len/(60)

2.1579520833333334